In [1]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [5]:
edata = pd.read_csv(r'C:\Users\hp\Documents\forbidden\Hidden_Databases\sms_raw_NB.csv', encoding = "ISO-8859-1")

In [7]:
## Cleaning data ##
stop_words = []
## Load custom built Stopwords
with open (r'C:\Users\hp\Documents\forbidden\Hidden_Databases\stop.txt','r') as sw:
    stop_words = sw.read()

In [8]:
stop_words = stop_words.split("\n") 
## spliting each string to convert to list ##

In [9]:
import re 
def cleaning_text(i):
    i = re.sub("[^A-Za-z" "]+", " ",i).lower()
## i = re.sub("[0-9" ""]+"," ",i)
    w = []
    for word in i.split(" "):
        if len(word) > 3:
            w.append(word)
    return (" ".join(w))

In [10]:
## testing above function with sample text => removes punctuations, numbers ##
cleaning_text("Hope you are having a good week.Just checking in")
cleaning_text("hope i can understand your feelings 123121. 123 hi hw .. are you?")
cleaning_text("Hi how are you, I am sad")

''

In [11]:
edata.text = edata.text.apply(cleaning_text)

In [12]:
## removing empty rows ##
edata = edata.loc[edata.text != "", :]

In [13]:
## split data into train and test data sets ##
from sklearn.model_selection import train_test_split

In [14]:
e_train, e_test = train_test_split(edata, test_size = 0.2)

In [15]:
## CountVectorizer, Convert a collection of text documents to a matrix of token counts ##
## Creating a matrix of token counts for the entire text document ##
def split_into_words(i):
    return [word for word in i.split(" ")]

In [16]:
## Defining the preparation of mail texts into word count matrix format - Bag of Words ##
emails_bow = CountVectorizer(analyzer = split_into_words).fit(edata.text)

In [17]:
## Defining BOW for all messages ##
all_emails_matrix = emails_bow.transform(edata.text)

In [18]:
## For training messages ##
train_emails_matrix = emails_bow.transform(e_train.text)

In [19]:
## For testing messages ##
test_emails_matrix = emails_bow.transform(e_test.text)

In [20]:
## Learning Term weighting and Normalizing on entire emails ##
tfidf_transformer = TfidfTransformer().fit(all_emails_matrix)

In [21]:
## Preparing TFIDF for train emails ##
train_tfidf = tfidf_transformer.transform(train_emails_matrix)
train_tfidf.shape

(4401, 6662)

In [22]:
## Preparing TFIDF for test emails ##
test_tfidf = tfidf_transformer.transform(test_emails_matrix)
test_tfidf.shape

(1101, 6662)

In [23]:
## Preparing a naive bayes model on training data set ##
from sklearn.naive_bayes import MultinomialNB as MB

In [24]:
classifier_mb = MB()
classifier_mb.fit(train_tfidf, e_train.type)

MultinomialNB()

In [25]:
## Evaluation on test data ##
test_pred_m = classifier_mb.predict(test_tfidf)

In [26]:
pd.crosstab(test_pred_m, e_test.type)

type,ham,spam
row_0,,
ham,951,36
spam,5,109


In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(test_pred_m, e_test.type)

0.9627611262488647